In [2]:
pip install rasterio

   ---------------------------------------- 0.0/25.7 MB ? eta -:--:--
   ---------------------------------------- 0.3/25.7 MB ? eta -:--:--
   - -------------------------------------- 1.0/25.7 MB 3.4 MB/s eta 0:00:08
   -- ------------------------------------- 1.3/25.7 MB 2.6 MB/s eta 0:00:10
   --- ------------------------------------ 2.1/25.7 MB 2.7 MB/s eta 0:00:09
   ---- ----------------------------------- 3.1/25.7 MB 3.2 MB/s eta 0:00:07
   ------ --------------------------------- 4.2/25.7 MB 3.6 MB/s eta 0:00:06
   -------- ------------------------------- 5.5/25.7 MB 3.9 MB/s eta 0:00:06
   --------- ------------------------------ 6.3/25.7 MB 3.9 MB/s eta 0:00:05
   ---------- ----------------------------- 6.8/25.7 MB 3.7 MB/s eta 0:00:06
   ------------ --------------------------- 8.1/25.7 MB 3.9 MB/s eta 0:00:05
   -------------- ------------------------- 9.4/25.7 MB 4.1 MB/s eta 0:00:04
   ---------------- ----------------------- 10.7/25.7 MB 4.3 MB/s eta 0:00:04
   ---------

In [4]:
import os
import numpy as np
import rasterio
from rasterio.enums import Resampling
from rasterio.warp import reproject


In [6]:
base_dir = "data/sentinal2/"

paths = {
    "date1": {
        "red": f"{base_dir}/date1/B04.jp2",
        "nir": f"{base_dir}/date1/B08.jp2",
        "scl": f"{base_dir}/date1/SCL.jp2",
    },
    "date2": {
        "red": f"{base_dir}/date2/B04.jp2",
        "nir": f"{base_dir}/date2/B08.jp2",
        "scl": f"{base_dir}/date2/SCL.jp2",
    }
}


In [7]:
def read_raster(path):
    with rasterio.open(path) as src:
        data = src.read(1).astype("float32")
        profile = src.profile
    return data, profile


In [8]:
def resample_to_match(source, source_profile, target_profile):
    resampled = np.empty(
        (target_profile["height"], target_profile["width"]),
        dtype=np.float32
    )

    reproject(
        source,
        resampled,
        src_transform=source_profile["transform"],
        src_crs=source_profile["crs"],
        dst_transform=target_profile["transform"],
        dst_crs=target_profile["crs"],
        resampling=Resampling.nearest
    )
    return resampled


In [9]:
def cloud_mask_ndvi(red, nir, scl):
    ndvi = (nir - red) / (nir + red + 1e-6)

    valid_mask = np.isin(scl, [4, 5])
    ndvi[~valid_mask] = np.nan

    return ndvi

The Scene Classification Layer (20 m) was resampled to match the 10 m spatial resolution of optical bands using nearest-neighbor resampling to preserve class integrity.

In [12]:
os.makedirs("output/ndvi", exist_ok=True)

for label in ["date1", "date2"]:
    print(f"\nProcessing {label}")

    red, red_profile = read_raster(paths[label]["red"])
    nir, _ = read_raster(paths[label]["nir"])
    scl, scl_profile = read_raster(paths[label]["scl"])

    scl_10m = resample_to_match(scl, scl_profile, red_profile)

    ndvi = cloud_mask_ndvi(red, nir, scl_10m)

    ndvi_profile = red_profile.copy()
    ndvi_profile.update(dtype="float32", count=1, nodata=np.nan)

    out_path = f"output/ndvi/ndvi_{label}.tif"

    with rasterio.open(out_path, "w", **ndvi_profile) as dst:
        dst.write(ndvi, 1)

    print(f"Saved → {out_path}")



Processing date1
Saved → output/ndvi/ndvi_date1.tif

Processing date2
Saved → output/ndvi/ndvi_date2.tif
